<a href="https://colab.research.google.com/github/Whoami666/HSE_Lab/blob/main/clean_models/SMOTE_schiz_models_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 3.8 MB/s eta 0:00:00


In [3]:
%cd drive
%cd MyDrive
%cd shiz

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/shiz


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [5]:
df = pd.read_excel('df_standard_cleaned_21feb.xlsx') 
df1d = pd.read_excel('df_standard_only1dfeatures.xlsx')

df = df.set_index('ID')
df1d = df1d.set_index('ID')
df.head(1)

,sex,tds,chroma_cens_avg,chroma_cens_std,chroma_cqt_avg,chroma_cqt_std,chroma_stft_avg,chroma_stft_std,melspectrogram_avg,melspectrogram_std,...,spectral_flatness_std,spectral_rolloff_avg,spectral_rolloff_std,tempogram_avg,tempogram_std,tonnetz_avg,tonnetz_std,zero_crossing_rate_avg,zero_crossing_rate_std,tds_binary
ID,,,,,,,,,,,,,,,,,,,,,
PD-003,1,0,0.685706,0.428088,0.425039,0.296394,0.339246,0.434412,0.327867,0.348849,...,0.31499,0.397736,0.254215,0.747885,0.719312,0.691479,0.329593,0.197304,0.281692,0


Models

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.inspection import permutation_importance

from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from sklearn.svm import SVC

In [7]:
from imblearn.over_sampling import SMOTE

In [8]:
def calculate_scores(model, givenx, giveny, xtest):
  kf = KFold(n_splits=10)
  crossval_f1 = cross_val_score(model, givenx, giveny, scoring='f1_macro', cv = kf, verbose=False)
  mean_score = crossval_f1.mean()
  std_score = crossval_f1.std()
  print('f1_macro', mean_score, '±', std_score)

  crossval_accuracy = cross_val_score(model, givenx, giveny, scoring='accuracy', cv = kf, verbose=False)
  mean_score = crossval_accuracy.mean()
  std_score = crossval_accuracy.std()
  print('accuracy', mean_score, '±', std_score)

  crossval_recall = cross_val_score(model, givenx, giveny, scoring='recall_macro', cv = kf, verbose=False)
  mean_score = crossval_recall.mean()
  std_score = crossval_recall.std()
  print('recall', mean_score, '±', std_score)

  if giveny.nunique() == 2:
    y_proba = model.predict_proba(xtest)
    print('roc_auc_score', roc_auc_score(y_test, y_proba[:,1]))

In [17]:
def svm_class(xtrain, ytrain, xtest, ytest, xgiven, ygiven):
  warnings.filterwarnings('ignore')
  opt = BayesSearchCV(
      SVC(),
      {
          'C': (1e-6, 1e+6, 'log-uniform'),
          'gamma': (1e-6, 1e+1, 'log-uniform'),
          'degree': (1, 8),  # integer valued parameter
          'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
      },
      scoring='f1_macro',
      n_iter=32,
      cv=3
  )

  opt.fit(xtrain, ytrain)

  print("val. score: %s" % opt.best_score_)
  print("test score: %s" % opt.score(xtest, ytest), '\n')


  params = opt.best_params_
  svm = SVC(gamma=params['gamma'], degree=params['degree'], C=params['C'], kernel=params['kernel'], probability=True)
  svm.fit(xtrain, ytrain)
  y_pred = svm.predict(xtest)
  print(classification_report(ytest,y_pred))

  calculate_scores(svm, xgiven, ygiven, xtest)

**Non-binary**

All features

In [18]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [19]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.9374862298853083
test score: 0.9531532722220648 

              precision    recall  f1-score   support

           0       1.00      0.85      0.92        55
           1       0.94      1.00      0.97        60
           2       0.94      1.00      0.97        65

    accuracy                           0.96       180
   macro avg       0.96      0.95      0.95       180
weighted avg       0.96      0.96      0.95       180

f1_macro 0.8173253663386959 ± 0.2626905016772233
accuracy 0.9533333333333334 ± 0.062420319578613964
recall 0.8473090380933519 ± 0.23379799298439763


Only1D

In [20]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [21]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.9122342383544612
test score: 0.940775166262112 

              precision    recall  f1-score   support

           0       1.00      0.82      0.90        55
           1       0.90      1.00      0.94        60
           2       0.96      1.00      0.98        65

    accuracy                           0.94       180
   macro avg       0.95      0.94      0.94       180
weighted avg       0.95      0.94      0.94       180

f1_macro 0.851390941434045 ± 0.20342482495173692
accuracy 0.9533333333333334 ± 0.06320650068145729
recall 0.8929134482075659 ± 0.1795308753298779


**Binary**

All features

In [22]:
X = df.drop(['tds', 'tds_binary'], axis=1)
y = df.tds_binary

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [23]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.9721276773908354
test score: 0.9832167832167833 

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        55
           1       0.98      0.98      0.98        65

    accuracy                           0.98       120
   macro avg       0.98      0.98      0.98       120
weighted avg       0.98      0.98      0.98       120

f1_macro 0.982239881920082 ± 0.043314987266555006
accuracy 0.99 ± 0.02134374745810949
recall 0.992962962962963 ± 0.014098419489388342
roc_auc_score 0.9997202797202798


Only 1d-features

In [24]:
X = df1d.drop(['tds', 'tds_binary'], axis=1)
y = df1d.tds_binary

oversample = SMOTE(k_neighbors=2)
X, y = oversample.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=16)

In [25]:
svm_class(X_train, y_train, X_test, y_test, X, y)

val. score: 0.8773589239721199
test score: 0.9230714438350309 

              precision    recall  f1-score   support

           0       1.00      0.84      0.91        55
           1       0.88      1.00      0.94        65

    accuracy                           0.93       120
   macro avg       0.94      0.92      0.92       120
weighted avg       0.93      0.93      0.92       120

f1_macro 0.8071049454264465 ± 0.2109432403789074
accuracy 0.93 ± 0.0752034278178565
recall 0.853778987699133 ± 0.2008191519763439
roc_auc_score 0.9745454545454546
